In [657]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from math import e

### Método forward

Se encarga de devolver el resultado de evaluar la función $f_\theta$.
$$f_\theta = W_2 \sigma (W_1 x + b_1)+b_2$$

In [658]:
def forward(W1, W2, b1, b2, x):
    capa1 = W1 @ x + b1
    
    sigmoide = 1/(1+np.exp(-capa1))

    capa2 = W2 @ sigmoide + b2
    return capa2

In [659]:
#Calculo del gradiente numerico
def funcion_objetivo(x, y, W1, W2, b1, b2):
    loss = 0.5*np.mean((forward(W1,W2,b1,b2,x) - y)**2)
    return loss

def numerical_gradient(x, y, W1, W2, b1, b2, eps):
    for i in range(W1.shape[0]):
        for j in range(W1.shape[1]):
            derivada_W1 np.zeros_like(W1)
            W1_izq = W1 
            W1_izq[i][j] = W1_izq[i][j] + eps
            W1_der = W1 
            W1_der[i][j] = W1_der[i][j] - eps
            izq_W1 = funcion_objetivo(x, y, W1_izq, W2, b1, b2)
            der_W1 = funcion_objetivo(x, y, W1_der, W2, b1, b2)
            derivada_W1[i, j] = (izq_W1 - der_W1)/(2*eps)
    for i in range(W2.shape[0]):
        for j in range(W2.shape[1]):
            W2_izq = W2
            W2_izq[i][j] = W2_izq[i][j] + eps
            W2_der = W2
            W2_der[i][j] = W2_der[i][j] - eps
            izq_W2 = funcion_objetivo(x, y, W1, W2_izq, b1, b2)
            der_W2 = funcion_objetivo(x, y, W1, W2_der, b1, b2)
            derivada_W2[i, j] = (izq_W2 - der_W2)/(2*eps)
    for i in range(b1.shape[0]):
        for j in range(b1.shape[1]):
            b1_izq = b1
            b1_izq[i][j] = b1_izq[i][j] + eps
            b1_der = b1
            b1_der[i][j] = b1_der[i][j] - eps
            izq_b1 = funcion_objetivo(x, y, W1, W2, b1_izq, b2)
            der_b1 = funcion_objetivo(x, y, W1, W2, b1_der, b2)
            derivada_b1[i, j] = (izq_b1 - der_b1)/(2*eps)
    for i in range(b2.shape[0]):
        for j in range(b2.shape[1]):
            b2_izq = b2
            b2_izq[i][j] = b2_izq[i][j] + eps
            b2_der = b2
            b2_der[i][j] = b2_der[i][j] - eps
            izq_b2 = funcion_objetivo(x, y, W1, W2, b1, b2_der)
            der_b2 = funcion_objetivo(x, y, W1, W2, b1, b2_izq)
            derivada_b2[i, j] = (izq_b2 - der_b2)/(2*eps)
    
    gradiente = np.zeros((4,1))
    gradiente[0] = derivada_W1
    gradiente[1] = derivada_W2
    gradiente[2] = derivada_b1
    gradiente[3] = derivada_b2
    
    return gradiente


In [660]:
# funcion fit y loop de entrenamiento
def fit(x, y, W1, W2, b1, b2, learning_rate, epochs):
    eps = 1e-3
    loss_accum = []
    j = 0
    for i in range(epochs):

        gradiente = numerical_gradient(x, y, W1, W2, b1, b2, eps)
        W1 = W1 - learning_rate*gradiente[0]
        W2 = W2 - learning_rate*gradiente[1]
        b1 = b1 - learning_rate*gradiente[2]
        b2 = b2 - learning_rate*gradiente[3]
        
        loss = funcion_objetivo(x, y, W1, W2, b1, b2)
        loss_accum.append(loss)

    theta = [W1, W2, b1, b2]
    return theta, loss_accum

In [661]:
def predict(x, W1, W2, b1, b2):
    y = forward(W1, W2, b1, b2, x)
    return y

### Inicialización de variables W1, b1, W2 y b2

In [662]:
W1 = np.random.random((5, 6))  # matriz 5x6 
b1 = np.random.random((5, 1))  # vector 5x1

W2 = np.random.random((1, 5))  # matriz 1x5
b2 = np.random.random((1, 1))  

### Recolección de datos

Tomamos los datos del archivo y los asignamos desde la posición 0 a la 315 como los de entrenamiento, y desde la 315 hasta el final como los datos de prueba.

In [663]:
df = pd.read_excel("Real estate valuation data set.xlsx")
datos = df.to_numpy()

X = datos[:100, [1,2,3,4,5,6]] 

Y = datos[:100,7]

X_test = datos[380:, [1,2,3,4,5,6]]
Y_test = datos[380:,7]

### Normalizamos las matrices

In [664]:
def normalizar(x):
    for i in range(x.shape[0]):
        x[i] = (x[i] - x[i].mean())/x[i].std()
    return x  

x_norm = normalizar(X)
x_test_norm = normalizar(X_test)

### Entrenamiento de la red

In [665]:
xt_norm = x_norm.transpose()

res = fit(xt_norm, Y, W1, W2, b1, b2, 0.0005, 1000)
theta = res[0]
loss = res[1]

xt_test_norm = x_test_norm.transpose()

Xt_test = X_test.transpose()

predictions = []
resta = []

preds = forward(theta[0], theta[1], theta[2], theta[3], xt_test_norm)

print(preds)
print(loss)

# for i in range(X_test.shape[0]): 
#     predictions.append(predict(xt_test_norm[:,i], theta[0], theta[1], theta[2], theta[3]))
#     resta.append(predictions[i] - Y_test[i])
#     print(predictions[i], "- ", Y_test[i], "=", resta[i])

# for i in range(X_test.shape[0]): 
#     predictions.append(predict(Xt_test[:,i], theta[0], theta[1], theta[2], theta[3]))
#     resta.append(predictions[i] - Y_test[i])
#     print(predictions[i], "- ", Y_test[i], "=", resta[i])


[[1.87193821 1.86277477 1.92814511 1.8836558  1.92762343 1.85971062
  1.86672555 1.9258717  1.92654792 1.86185989 1.87630035 1.92452477
  1.88009411 1.90427909 1.92777525 1.88402463 1.90278678 1.91072644
  1.92221294 1.86561812 1.88235225 1.92614112 1.86660198 1.8641646
  1.87191603 1.86277967 1.87641898 1.92724027 1.92635184 1.92757121
  1.8604072  1.87731478 1.86157685 1.86039706]]
[758.0863201072519, 758.0863201072519, 758.0863201072519, 758.0863201072519, 758.0863201072519, 758.0863201072519, 758.0863201072519, 758.0863201072519, 758.0863201072519, 758.0863201072519, 758.0863201072519, 758.0863201072519, 758.0863201072519, 758.0863201072519, 758.0863201072519, 758.0863201072519, 758.0863201072519, 758.0863201072519, 758.0863201072519, 758.0863201072519, 758.0863201072519, 758.0863201072519, 758.0863201072519, 758.0863201072519, 758.0863201072519, 758.0863201072519, 758.0863201072519, 758.0863201072519, 758.0863201072519, 758.0863201072519, 758.0863201072519, 758.0863201072519, 758.

In [666]:
# data = {'epochs': range(1000),
#         'loss_acum': loss_acum}

# df = pd.DataFrame(data)

# df.plot(x='epochs', y='loss_acum')

# plt.title('Funcion objetivo a lo largo del entrenamiento')
# plt.xlabel('Epochs')
# plt.ylabel('Loss Acumulated')

# plt.show()